## Set Up

In [ ]:
import pandas as pd
import json
import requests
import urllib

Importing pandas is making the data being able to be read in the format of a data table.Importing json is the type of file format we are using. Importing requests is importing the ability for Jupyter Notebook to take the query and access the data on Twitter. Lastly, importing the urllib is importing the python library so that any code ran in python applies to Jupyter Notebook.¶

In [ ]:
ls

In [ ]:
bearer_token = pd.read_csv("Twitter_keys_922.txt", header = 0, sep ='\t')

The bearer token acts as the keys to a house. Instead of listing the tokens here in the document, the code is saying, in order to access the bearer token, it can be found in this text document.¶

In [ ]:
bearer_token['Bearer Token'].iloc[0]

From the text file, the code is saying to read the 0th term and that is the bearer token.

In [ ]:
headers = {'Authorization': 'Bearer {}'.format(bearer_token['Bearer Token'].iloc[0])}

Headers is making itself equal the authorization of the bearer token.

In [ ]:
endpoint = 'https://api.twitter.com/2/tweets/search/recent'

The endpoint is the url to where the requests will be taken from.

## Defining Query and Query Parameters

In [ ]:
user_fields = 'username'

In [ ]:
tweet_fields = 'author_id,lang,public_metrics,created_at'

In [ ]:
query_text = '(autism ADHD) lang:en -is:retweet'

For this assignemnt, I wanted to research what is being said on Twitter about autism and ADHD. Both neurodivergencies overlap a lot and I was curious whether or not the tweets would be more about personal experiences or other topics including adhd and autism. My limitations with this query are I would like to narrow down more on the topic specifically of females with adhd and or autism. I had a hard time doing so as well as making the query functional. Another limitation was at first making my query too specfic. I would like to gather data on a specific topic and by doing this I think that I would need to collect the data multiple times and or run different functions with the query to collect the amount of responses I need. I did not get a chance to understand how to collect the 300 responses, but I did collect around 200. 
Above the query,the query parameters are being established. Within the query the following things will be included: username, author, language, time created,
The author id helps to identify the user, lang is the langauge of the tweet, time created is the time the tweet was created, and the public metrics is accessed at the time that the query is accessed.¶

In [ ]:
query_url = endpoint + '?query={}&tweet.fields={}&max_results=100'.format(query_text,tweet_fields,user_fields)

In [ ]:
query_url

In [ ]:
response = requests.get(query_url, headers = headers)

Assigning the response to the twitter API requests.

## First set of responses

In [ ]:
response.text

In [ ]:
response_dict = json.loads(response.text)

Assigning the response dictionary to be the responses of the query.¶

In [ ]:
response_dict.keys()

Listing the keys listed in the response dictionary.

In [ ]:
response_dict['data']

Listing the responses in a more orderly way.

In [ ]:
response_dict['meta']

Listing the result counts that are found within the query and its parameters as well as the newest tweet and oldest tweet.¶

In [ ]:
response_df =pd.DataFrame(response_dict['data'])

The response data frame is being assigned to the response data of the query.¶

In [ ]:
response_df.head()

Even though that this data table helps visualize the data more, I would have liked to learn and understand how to dive into the data even further. I would have liked to learn how to remove certain columns from the dataframe as well as from certain columns, expand on that data and extract things such as the usernames and the text itself. It would have been helpful to learn how to make the data text from the tweets, appear in full.

In [ ]:
public_metrics_df = pd.DataFrame(list(response_df['public_metrics']))

In [ ]:
response_final = pd.concat([response_df, public_metrics_df])

## Second set of responses

In [ ]:
next_query_url = query_url + "&next_token={}".format(response_dict['meta']['next_token'])

In [ ]:
next_response = requests.get(next_query_url, headers = headers)

The line of code accesses the next page of tweets that are generated from the query.¶

In [ ]:
next_response.text

Next page of tweets

In [ ]:
next_response_dict = json.loads(next_response.text)

Assigning the next dictionary of responses to the next page of responses.

## Setting up a Loop

In [ ]:
def twt_recent_search (query, num_pages, headers):
    response_list = []
    next_token = ''
    for i in range(0, num_pages):
        if i > 0:
            this_query = query + "&next_token={}".format(next_token)
            
        else:
            this_query = query
            
        this_response = requests.get(this_query, headers = headers)
        print(this_response.status_code)
        this_response_dict = json.loads(this_response.text)
        response_list.append(this_response_dict)
        next_token = this_response_dict ['meta']['next_token']
    
    
    
    
    return response_list

This long cell is defining that the recent search will include the query parameters and will result in a certain number of pages. 
i is representing the number of pages. As long as i is greater than 0, then more responses will be received. If not, the query will remain the same and will not go on to a next page. 
this response is assigning the requests and the query to and in turn, will print the response status code if all the parameters are correct within them. From there, then the responses will be listed as the next token or the next page of responses. I believe that I understand what this loop is doing, however I am not quite sure of its importance. 

In [ ]:
my_responses = twt_recent_search (query_url, 4, headers)

4 represents the number of pages needed in order to collect all the data I need. I would have liked to learn how to be able to request 4 pages of the data and be able to see it visualized. 

## Start of the results from the loop

In [ ]:
results_1 = pd.DataFrame.from_records(my_responses)

In [ ]:
my_responses

In [ ]:
data_list = list(results_1['data'])

The data list is assigning the list of results and from that list of results, specifically the data key. 

In [ ]:
results_1

In [ ]:
data_list_of_dfs = [pd.DataFrame(x) for x in data_list]

The data list of data frames is being assigned to the Pandas Dataframe of x. I am not entirely sure what the x represents in this case. 

In [ ]:
data_df= pd.concat(data_list_of_dfs)

The data data frame is being assigned to the pandas concat data list of dataframes. I am not sure what concat means or what the specific function is. I looked up the term and I am still confused. 

In [ ]:
data_df.head()

## Conclusion

Overall, this assignment was quite difficult for me in terms of understanding the code. I learned that I need to start asking more questions in class if I am confused or at the very least to clarify my understanding of the code and its importance and function. I would have liked to been able to get to the point of starting to analyze my data to see what kind of tweets exist that specifically pertain to women with adhd and or autism. 